## Tuning Page + Backend

In [1]:
import ipyvuetify as v
from tuning_page import TuningPage
from pidtuner import PIDTuner

In [9]:
class Tuning(TuningPage):
    def __init__(self):
        super().__init__()
        
        self.pid = PIDTuner()
        
        self.pid.k_p = 2
        self.pid.tau_p = 10
        self.pid.theta_p = 0
        
        self.pid.k_c, self.pid.tau_i, self.pid.tau_d = self.pid.rule_based(method='IMC')
        
        self.pid_card.p_slider.set_limits(min(0.1*self.pid.k_c, 1), 10*self.pid.k_c, self.pid.k_c)
        self.pid_card.i_slider.set_limits(0.5*self.pid.tau_i, 1.5*self.pid.tau_i, self.pid.tau_i)
        self.pid_card.d_slider.set_limits(0.5*self.pid.tau_d, max(1.5*self.pid.tau_d, self.pid.tau_d+5), self.pid.tau_d)
        
        self.model_card.g_slider.set_limits(0.5*self.pid.k_p, 1.5*self.pid.k_p, self.pid.k_p)
        self.model_card.tau_slider.set_limits(0.5*self.pid.tau_p, 1.5*self.pid.tau_p, self.pid.tau_p)
        self.model_card.d_slider.set_limits(0, self.pid.theta_p+5, self.pid.theta_p)
        
        self.pid_card.p_slider.slider.on_event('change', self.update_kc)
        self.pid_card.i_slider.slider.on_event('change', self.update_taui)
        self.pid_card.d_slider.slider.on_event('change', self.update_taud)
        
        self.model_card.g_slider.slider.on_event('change', self.update_kp)
        self.model_card.tau_slider.slider.on_event('change', self.update_taup)
        self.model_card.d_slider.slider.on_event('change', self.update_thetap)
        
    def update_kp(self, item, *args):
        self.pid.k_p = item.v_model
        self.run_pid()
        
    def update_taup(self, item, *args):
        self.pid.tau_p = item.v_model
        self.run_pid()
    
    def update_thetap(self, item, *args):
        self.pid.theta_p = item.v_model
        self.run_pid()
        
    def update_kc(self, item, *args):
        self.pid.k_c = item.v_model
        self.run_pid()
        
    def update_taui(self, item, *args):
        self.pid.tau_i = item.v_model
        self.run_pid()
    
    def update_taud(self, item, *args):
        self.pid.tau_d = item.v_model
        self.run_pid()
        
    def run_pid(self):
        pv, op, t = self.pid.control_sim()
        
        self.pv_card.update_plot(t, pv)
        self.op_card.update_plot(t, op)

In [10]:
a = Tuning()
a

Tuning(children=[Container(children=[CustomData(children=[CardTitle(children=[Spacer(), 'Tuning Constraints', …